<a href="https://colab.research.google.com/github/aravindchakravarti/Deep_Learning_EVA_4/blob/master/S4_MNIST_PyTorch/EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
# Import core module
import torch.nn as nn
# We need support for functional programming
import torch.nn.functional as F
import torch.optim as optim
# For MNIST dataset
from torchvision import datasets, transforms
# For Global Average Pooling
from torch.nn import AvgPool2d

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1)      #I=28, O=28, RF=3
        self.batch1 = nn.BatchNorm2d(num_features=8)    
        self.conv2 = nn.Conv2d(8, 8, 3, padding=1)      #I=28, O=28, RF=5
        self.batch2 = nn.BatchNorm2d(num_features=8)    
        self.conv3 = nn.Conv2d(8, 12, 3, padding=1)     #I=28, O=28, RF=7
        self.batch3 = nn.BatchNorm2d(num_features=12)   
        self.pool1 = nn.MaxPool2d(2, 2)                 #I=28, O=14, RF=14
        self.conv4 = nn.Conv2d(12, 10, 1)

        self.conv5 = nn.Conv2d(10, 16, 3, padding=1)    #I=14, O=14, RF=17
        self.batch4 = nn.BatchNorm2d(num_features=16)
        self.conv6 = nn.Conv2d(16, 16, 3, padding=1)    #I=14, O=14, RF=20
        self.batch5 = nn.BatchNorm2d(num_features=16)
        self.conv7 = nn.Conv2d(16, 16, 3, padding=1)    #I=14, O=14, RF=23
        self.batch6 = nn.BatchNorm2d(num_features=16)
        self.pool2 = nn.MaxPool2d(2, 2)                 #I=14, O=07, RF=26
        self.conv8 = nn.Conv2d(16, 10, 1)

        self.conv9 = nn.Conv2d(10, 32, 3, padding=0)    #I=07, O=05, RF=29
        self.batch7 = nn.BatchNorm2d(num_features=32)
        self.conv10 = nn.Conv2d(32, 10, 3, padding=0)   #I=05, O=03, RF=32
        self.batch8 = nn.BatchNorm2d(num_features=10)
        self.avg_pool = nn.AvgPool2d(kernel_size=3)

    def forward(self, x):
        x = self.batch1(F.relu(self.conv1(x)))        #See above for Input\
        x = self.batch2(F.relu(self.conv2(x)))        #Output and Receptive Field
        x = self.batch3(F.relu(self.conv3(x)))
        x = self.pool1(x)
        x = F.relu(self.conv4(x))

        x = self.batch4(F.relu(self.conv5(x)))
        x = self.batch5(F.relu(self.conv6(x)))
        x = self.batch6(F.relu(self.conv7(x)))
        x = self.pool2(x)
        x = F.relu(self.conv8(x))

        x = self.batch7(F.relu(self.conv9(x)))
        x = self.batch8(F.relu(self.conv10(x)))
        x = self.avg_pool(x)
        x = x.view(-1, 10)                           # Don't want 10x1x1..
        return F.log_softmax(x)

In [26]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3            [-1, 8, 28, 28]             584
       BatchNorm2d-4            [-1, 8, 28, 28]              16
            Conv2d-5           [-1, 12, 28, 28]             876
       BatchNorm2d-6           [-1, 12, 28, 28]              24
         MaxPool2d-7           [-1, 12, 14, 14]               0
            Conv2d-8           [-1, 10, 14, 14]             130
            Conv2d-9           [-1, 16, 14, 14]           1,456
      BatchNorm2d-10           [-1, 16, 14, 14]              32
           Conv2d-11           [-1, 16, 14, 14]           2,320
      BatchNorm2d-12           [-1, 16, 14, 14]              32
           Conv2d-13           [-1, 16, 14, 14]           2,320
      BatchNorm2d-14           [-1, 16,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:73: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    # Disabling TQDM. Don't want to hurt CPU on my machine
    #pbar = tqdm(train_loader)
    #       If you enable TQDM, replace to pbar-- below
    #                                           |
    #                                           ▼
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        #pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [29]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:73: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Test set: Average loss: 0.0944, Accuracy: 9794/10000 (97.94%)


Test set: Average loss: 0.0673, Accuracy: 9841/10000 (98.41%)


Test set: Average loss: 0.0711, Accuracy: 9797/10000 (97.97%)


Test set: Average loss: 0.0445, Accuracy: 9897/10000 (98.97%)


Test set: Average loss: 0.0302, Accuracy: 9912/10000 (99.12%)


Test set: Average loss: 0.0388, Accuracy: 9891/10000 (98.91%)


Test set: Average loss: 0.0256, Accuracy: 9927/10000 (99.27%)


Test set: Average loss: 0.0432, Accuracy: 9877/10000 (98.77%)


Test set: Average loss: 0.0296, Accuracy: 9923/10000 (99.23%)


Test set: Average loss: 0.0254, Accuracy: 9926/10000 (99.26%)


Test set: Average loss: 0.0259, Accuracy: 9927/10000 (99.27%)


Test set: Average loss: 0.0206, Accuracy: 9943/10000 (99.43%)


Test set: Average loss: 0.0282, Accuracy: 9920/10000 (99.20%)


Test set: Average loss: 0.0247, Accuracy: 9929/10000 (99.29%)


Test set: Average loss: 0.0255, Accuracy: 9923/10000 (99.23%)


Test set: Average loss: 0.0238, Accurac